In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/2021-spring-programming lab/aoi/train_images.zip'

In [ ]:
from pandas import read_csv
import pandas as pd
# load file as CSV
from sklearn.preprocessing import LabelBinarizer
import numpy
import numpy as np
filename = '/content/drive/MyDrive/2021-spring-programming lab/aoi/train.csv'
mapping1_csv = read_csv(filename,encoding="Big5")
filename = '/content/drive/MyDrive/2021-spring-programming lab/aoi/test.csv'
mapping2_csv = read_csv(filename,encoding="Big5")
mapping1_csv = mapping1_csv.values.tolist()
mapping2_csv = mapping2_csv.values.tolist()

import cv2
import os
from PIL import Image
from numpy import savez_compressed
from numpy import asarray

def load_dataset(directory,mapping_csv):
    X, y = list(), list()
    data = []
    labels = []
    i=0
    # loop over the image paths
    for imagePath, label in mapping_csv:
        image = Image.open(directory + imagePath)
        image = image.convert('RGB')
        image = image.resize((224,224))
        image = asarray(image)/255  # RGB -> x/255
        data.append(image)
        labels.append(label)
    # convert the data and labels to NumPy arrays while scaling the pixel
    # perform one-hot encoding on the labels
    from keras import utils as np_utils
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    X.extend(data)
    y.extend(labels)
    return asarray(X), asarray(y)

X, y = load_dataset('train_images/',mapping1_csv)
print(X.shape, y.shape)

In [ ]:
mapping1_csv

In [ ]:
# Train and Validation 8:2

from sklearn.model_selection import train_test_split
trainX,testX,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=0)
print(trainX.shape,testX.shape,trainy.shape,testy.shape)

In [ ]:
# Save the data temporarily in content then restart the runtime to avoid overusing RAM

np.save('trainX.npy',trainX)
np.save('trainy.npy',trainy)
np.save('testX.npy',testX)
np.save('testy.npy',testy)

In [ ]:
# reload the data from content

import numpy as np

trainX = np.load('/content/trainX.npy')
trainy = np.load('/content/trainy.npy')
testX = np.load('/content/testX.npy')
testy = np.load('/content/testy.npy')
print(trainX.shape,testX.shape,trainy.shape,testy.shape)

In [ ]:
# Set up for TPU 

# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from keras.layers import Dropout
from tensorflow.keras.applications import Xception

In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):

    conv_base = Xception(include_top=False, # fine-tune; removing top
                     weights='imagenet',    # pattern based on imagenet
                     input_shape=input_shape)
    
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]: # not involved in training
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
            
    top_model = conv_base.output
    # top_model = Dropout(0.2)(top_model)
    top_model = Flatten(name="flatten")(top_model)
    # top_model = Dropout(0.2)(top_model)
    # top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    # top_model = Dense(536, activation='relu')(top_model)
    top_model = Dense(268, activation='relu')(top_model)
    top_model = Dense(134, activation='relu')(top_model)
    top_model = Dense(67, activation='relu')(top_model)
    top_model = Dense(33, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
nadam = Nadam(0.001)

# with tpu_strategy.scope():
#   model=create_model((224,224,3),6,optimizer=nadam,fine_tune=3)

class MyThresholdCallback(tf.keras.callbacks.Callback): # self-defining callback: when loss <= 0.01
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        loss = logs["loss"]
        if loss <= self.threshold:
            self.model.stop_training = True
callback=MyThresholdCallback(threshold=0.01)

In [ ]:
model=create_model((224,224,3),6,optimizer='rmsprop',fine_tune=1)

In [ ]:
# model.summary()

In [ ]:
model.fit(trainX, trainy , batch_size=20, epochs=100,callbacks=[callback])

In [ ]:
model.save('/content/drive/MyDrive/2021-spring-programming lab/model_Xception_11.h5')

In [ ]:
model.evaluate(trainX,trainy)

In [ ]:
model.evaluate(testX,testy)